# Using Sample-Level Pipeline

## A sample-level (patient-level) visualization of the gene and cell-type of interest. 

In [ ]:
from genereporter.cell_pipeline import SamplePipeline

sp = SamplePipeline("/Users/samibening/Projects/genereporter/", "data/output/adata.h5ad")
adata = sp.get_adata()
adata